## Imports

In [12]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

## Q1. Embedding the query

In [2]:
query = 'I just discovered the course. Can I join now?'
model=TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

q_embedded = list(model.embed(query))[0]
print(f"Minimal value in the array: {min(q_embedded):.2f}")

Minimal value in the array: -0.12


## Q2. Cosine similarity with another vector

In [3]:
doc = 'Can I still join the course after the start date?'
d_embedded = list(model.embed(doc))[0]

cosine_similarity = q_embedded.dot(d_embedded)
print(f"Cosine similarity between the query and document: {cosine_similarity:.1f}")

Cosine similarity between the query and document: 0.9


## Q3. Ranking by cosine

In [4]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]


In [5]:
V = []

for document in documents:
    text = list(model.embed(document['text']))
    V.append(text[0].dot(q_embedded))

maxVal = max(V)
maxIndex = V.index(maxVal)
print(f"Most similar document: {maxIndex}\n {documents[maxIndex]['text']}")

Most similar document: 1
 Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


# Q4. Ranking by cosine, version two

In [8]:
V = []

for document in documents:
    full_text = document['question'] + ' ' + document['text']
    text = list(model.embed(full_text))
    V.append(text[0].dot(q_embedded))

maxVal = max(V)
maxIndex = V.index(maxVal)
print(f"Most similar document: {maxIndex}\n {documents[maxIndex]['question']}\n {documents[maxIndex]['text']}")

Most similar document: 0
 Course - Can I still join the course after the start date?
 Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


## Q5. Selecting the embedding model

In [10]:
min_dim = 1000
for model in TextEmbedding.list_supported_models():
    min_dim = min(min_dim, model["dim"])

print(f"Minimum Dimensionality: {min_dim}")

Minimum Dimensionality: 384


## Q6. Indexing with qdrant

In [18]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
client = QdrantClient("http://localhost:6333")

collection_name = "llm-homework"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [20]:
model='BAAI/bge-small-en'
points = []
id = 0

for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=full_text, model=model),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "question": doc['question'],
            "course": doc['course']
        }
    )
    points.append(point)

    id += 1

In [21]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
def search(query, limit=10):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [31]:
results = search("I just discovered the course. Can I join now?")
print(results)

points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_va